In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 2
import sys
import traceback # for stack trace formatting
import importlib
from pathlib import Path
from benedict import benedict
import numpy as np
import pandas as pd
# from pandas_profiling import ProfileReport ## for dataframe viewing

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
from benedict import benedict # https://github.com/fabiocaccamo/python-benedict#usage

# Pho's Formatting Preferences
# from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
# set_pho_preferences_concise()

## Pho's Custom Libraries:
from pyphocorehelpers.general_helpers import CodeConversion
from pyphocorehelpers.print_helpers import print_keys_if_possible, print_value_overview_only, document_active_variables, objsize, print_object_memory_usage, debug_dump_object_member_shapes, TypePrintMode
from pyphocorehelpers.print_helpers import get_now_day_str, get_now_time_str, get_now_time_precise_str

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths

# from PendingNotebookCode import _perform_batch_plot, _build_batch_plot_kwargs
from pyphoplacecellanalysis.General.NonInteractiveWrapper import batch_load_session, batch_extended_computations, SessionBatchProgress, batch_programmatic_figures, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme

session_batch_status = {}
session_batch_errors = {}
enable_saving_to_disk = False

global_data_root_parent_path = Path(r'W:\Data') # Windows Apogee
# global_data_root_parent_path = Path(r'/media/MAX/Data') # Diba Lab Workstation Linux
# global_data_root_parent_path = Path(r'/Volumes/MoverNew/data') # rMBP
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL\TESTING\Logging\debug_com.PhoHale.Spike3D.pipeline.log


# Load Pipeline

In [2]:
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'

## Data must be pre-processed using the MATLAB script located here: 
#     neuropy/data_session_pre_processing_scripts/KDIBA/IIDataMat_Export_ToPython_2022_08_01.m
# From pre-computed .mat files:

local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=self.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

## Animal `gor01`:
local_session_parent_context = local_session_root_parent_context.adding_context(collision_prefix='animal', animal='gor01', exper_name='one') # IdentifyingContext<('kdiba', 'gor01', 'one')>
local_session_parent_path = local_session_root_parent_path.joinpath(local_session_parent_context.animal, local_session_parent_context.exper_name) # 'gor01', 'one'
local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=['PhoHelpers', 'Spike3D-Minimal-Test', 'Unused'])

# local_session_parent_context = local_session_root_parent_context.adding_context(collision_prefix='animal', animal='gor01', exper_name='two')
# local_session_parent_path = local_session_root_parent_path.joinpath(local_session_parent_context.animal, local_session_parent_context.exper_name)
# local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=[])

### Animal `vvp01`:
# local_session_parent_context = local_session_root_parent_context.adding_context(collision_prefix='animal', animal='vvp01', exper_name='one')
# local_session_parent_path = local_session_root_parent_path.joinpath(local_session_parent_context.animal, local_session_parent_context.exper_name)
# local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=[])

# local_session_parent_context = local_session_root_parent_context.adding_context(collision_prefix='animal', animal='vvp01', exper_name='two')
# local_session_parent_path = local_session_root_parent_path.joinpath(local_session_parent_context.animal, local_session_parent_context.exper_name)
# local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=[])

# ### Animal `pin01`:
# local_session_parent_context = local_session_root_parent_context.adding_context(collision_prefix='animal', animal='pin01', exper_name='one')
# local_session_parent_path = local_session_root_parent_path.joinpath(local_session_parent_context.animal, local_session_parent_context.exper_name) # no exper_name ('one' or 'two') folders for this animal.
# local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=['redundant','showclus','sleep','tmaze'])

## Build session contexts list:
local_session_contexts_list = [local_session_parent_context.adding_context(collision_prefix='sess', session_name=a_name) for a_name in local_session_names_list] # [IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-07_11-26-53')>, ..., IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-13_14-42-6')>]

## Initialize `session_batch_status` with the NOT_STARTED status if it doesn't already have a different status
for curr_session_basedir in local_session_paths_list:
    curr_session_status = session_batch_status.get(curr_session_basedir, None)
    if curr_session_status is None:
        session_batch_status[curr_session_basedir] = SessionBatchProgress.NOT_STARTED # set to not started if not present
        # session_batch_status[curr_session_basedir] = SessionBatchProgress.COMPLETED # set to not started if not present

session_batch_status

local_session_names_list: ['2006-6-07_11-26-53', '2006-6-08_14-26-15', '2006-6-09_1-22-43', '2006-6-09_3-23-37', '2006-6-12_15-55-31', '2006-6-13_14-42-6']


{WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-07_11-26-53'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>,
 WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>,
 WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>,
 WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-09_3-23-37'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>,
 WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-12_15-55-31'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>,
 WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-13_14-42-6'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>}

In [3]:
%pdb off
# %%viztracer
basedir = local_session_paths_list[1] # NOT 3
print(f'basedir: {str(basedir)}')

# ==================================================================================================================== #
# Load Pipeline                                                                                                        #
# ==================================================================================================================== #
# epoch_name_whitelist = ['maze']
epoch_name_whitelist = None
active_computation_functions_name_whitelist=['_perform_baseline_placefield_computation', '_perform_time_dependent_placefield_computation', '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        # '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_time_dependent_pf_sequential_surprise_computation'
                                        '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding'
                                    ]
curr_active_pipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_whitelist=epoch_name_whitelist,
                                          computation_functions_name_whitelist=active_computation_functions_name_whitelist,
                                          saving_mode=PipelineSavingScheme.SKIP_SAVING, force_reload=False, skip_extended_batch_computations=True, debug_print=False, fail_on_exception=True)

Automatic pdb calling has been turned OFF
basedir: W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15
Loading loaded session pickle file results : W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\loadedSessPickle.pkl... done.
Loading pickled pipeline success: W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\loadedSessPickle.pkl.
property already present in pickled version. No need to save.
using provided computation_functions_name_whitelist: ['_perform_baseline_placefield_computation', '_perform_time_dependent_placefield_computation', '_perform_extended_statistics_computation', '_perform_position_decoding_computation', '_perform_firing_rate_trends_computation', '_perform_two_step_position_decoding_computation']
	 TODO: this will prevent recomputation even when the blacklist/whitelist or computation function definitions change. Rework so that this is smarter.
updating computation_results...
done.
	 TODO: this will prevent recomputation even when the blacklist/whitelist or computation function definitions c

# 2023-02-24 Decoders 
- [ ] where are cells chosen for inclusion in the input of the decoder?

In [4]:
long_epoch_name, short_epoch_name, global_epoch_name = long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]

recalculate_anyway = False

In [5]:
# Make the 1D Placefields and Decoders conform between the long and the short epochs:
long_pf1D = long_results.pf1D
short_pf1D = short_results.pf1D
global_pf1D = global_results.pf1D

# short_pf1D, did_update_bins = short_pf1D.conform_to_position_bins(long_pf1D, force_recompute=True) # not needed because it's done in one_step_decoder_1D.conform_to_position_bins(...)
long_one_step_decoder_1D, short_one_step_decoder_1D  = [results_data.get('pf1D_Decoder', None) for results_data in (long_results, short_results)]
short_one_step_decoder_1D, did_recompute = short_one_step_decoder_1D.conform_to_position_bins(long_one_step_decoder_1D, force_recompute=True)

## Build or get the two-step decoders for both the long and short:
long_two_step_decoder_1D, short_two_step_decoder_1D  = [results_data.get('pf1D_TwoStepDecoder', None) for results_data in (long_results, short_results)]
if recalculate_anyway or did_recompute or (long_two_step_decoder_1D is None) or (short_two_step_decoder_1D is None):
    curr_active_pipeline.perform_specific_computation(computation_functions_name_whitelist=['_perform_two_step_position_decoding_computation'], computation_kwargs_list=[dict(ndim=1)], enabled_filter_names=[long_epoch_name, short_epoch_name], fail_on_exception=True, debug_print=True)
    long_two_step_decoder_1D, short_two_step_decoder_1D  = [results_data.get('pf1D_TwoStepDecoder', None) for results_data in (long_results, short_results)]
    assert (long_two_step_decoder_1D is not None and short_two_step_decoder_1D is not None)

decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333
# decoding_time_bin_size = 0.03 # 0.03333333333333333
print(f'decoding_time_bin_size: {decoding_time_bin_size}')

self will be re-binned to match target_pf1D...
done.
self will be re-binned to match target_one_step_decoder...
Performing run_specific_computations_single_context on filtered_session with filter named "maze1"...
Performing _execute_computation_functions(...) with 1 registered_computation_functions...
np.isscalar(bin_size): 1.607897707662558
prev_one_step_bayesian_decoder.ndim == 1, so using [0.0] as active_ybin_centers.
two_step_decoder_result['most_likely_position_flat_max_likelihood_values'].shape = (36668,)
_execute_computation_functions(...): 
	accumulated_errors: None
Performing run_specific_computations_single_context on filtered_session with filter named "maze2"...
Performing _execute_computation_functions(...) with 1 registered_computation_functions...
np.isscalar(bin_size): 1.607897707662558
prev_one_step_bayesian_decoder.ndim == 1, so using [0.0] as active_ybin_centers.
two_step_decoder_result['most_likely_position_flat_max_likelihood_values'].shape = (26242,)
_execute_compu

#### Get 2D Decoders for validation and comparisons:

In [6]:
# Make the 2D Placefields and Decoders conform between the long and the short epochs:
long_pf2D = long_results.pf2D
short_pf2D = short_results.pf2D
global_pf2D = global_results.pf2D

# long_one_step_decoder_2D, short_one_step_decoder_2D  = [results_data.get('pf2D_Decoder', None) for results_data in (long_results, short_results)]
# long_two_step_decoder_2D, short_two_step_decoder_2D  = [results_data.get('pf2D_TwoStepDecoder', None) for results_data in (long_results, short_results)]

# short_pf2D, did_update_bins = short_pf2D.conform_to_position_bins(long_pf2D)
long_one_step_decoder_2D, short_one_step_decoder_2D  = [results_data.get('pf2D_Decoder', None) for results_data in (long_results, short_results)]
short_one_step_decoder_2D, did_recompute = short_one_step_decoder_2D.conform_to_position_bins(long_one_step_decoder_2D)

## Build or get the two-step decoders for both the long and short:
long_two_step_decoder_2D, short_two_step_decoder_2D  = [results_data.get('pf2D_TwoStepDecoder', None) for results_data in (long_results, short_results)]
if recalculate_anyway or did_recompute or (long_two_step_decoder_2D is None) or (short_two_step_decoder_2D is None):
    curr_active_pipeline.perform_specific_computation(computation_functions_name_whitelist=['_perform_two_step_position_decoding_computation'], computation_kwargs_list=[dict(ndim=1)], enabled_filter_names=[long_epoch_name, short_epoch_name], fail_on_exception=True, debug_print=True)
    long_two_step_decoder_2D, short_two_step_decoder_2D  = [results_data.get('pf2D_TwoStepDecoder', None) for results_data in (long_results, short_results)]
    assert (long_two_step_decoder_2D is not None and short_two_step_decoder_2D is not None)

self will be re-binned to match target_pf1D...
done.
self will be re-binned to match target_one_step_decoder...
Performing run_specific_computations_single_context on filtered_session with filter named "maze1"...
Performing _execute_computation_functions(...) with 1 registered_computation_functions...
np.isscalar(bin_size): 1.607897707662558
prev_one_step_bayesian_decoder.ndim == 1, so using [0.0] as active_ybin_centers.
two_step_decoder_result['most_likely_position_flat_max_likelihood_values'].shape = (36668,)
_execute_computation_functions(...): 
	accumulated_errors: None
Performing run_specific_computations_single_context on filtered_session with filter named "maze2"...
Performing _execute_computation_functions(...) with 1 registered_computation_functions...
np.isscalar(bin_size): 1.607897707662558
prev_one_step_bayesian_decoder.ndim == 1, so using [0.0] as active_ybin_centers.
two_step_decoder_result['most_likely_position_flat_max_likelihood_values'].shape = (26242,)
_execute_compu

AssertionError: 

In [ ]:
# Sums are similar:
print(f'{np.sum(long_one_step_decoder_2D.marginal.x.p_x_given_n) =},\t {np.sum(long_one_step_decoder_1D.p_x_given_n) = }') # 31181.999999999996 vs 31181.99999999999

## Validate:
assert long_one_step_decoder_2D.marginal.x.p_x_given_n.shape == long_one_step_decoder_1D.p_x_given_n.shape, f"Must equal but: {long_one_step_decoder_2D.marginal.x.p_x_given_n.shape =} and {long_one_step_decoder_1D.p_x_given_n.shape =}"
assert long_one_step_decoder_2D.marginal.x.most_likely_positions_1D.shape == long_one_step_decoder_1D.most_likely_positions.shape, f"Must equal but: {long_one_step_decoder_2D.marginal.x.most_likely_positions_1D.shape =} and {long_one_step_decoder_1D.most_likely_positions.shape =}"

## validate values:
assert np.allclose(long_one_step_decoder_2D.marginal.x.p_x_given_n, long_one_step_decoder_1D.p_x_given_n), f"1D Decoder should have an x-posterior equal to its own posterior"
assert np.allclose(curr_epoch_result['marginal_x']['most_likely_positions_1D'], curr_epoch_result['most_likely_positions']), f"1D Decoder should have an x-posterior with most_likely_positions_1D equal to its own most_likely_positions"

In [ ]:
# Gate the spikes that are coming in to build the decoder... those are from the placefields actually.
    # so it'll be gating the placefield cells.
    
active_config_name = 'maze1'
active_pf_1D = long_pf1D
# curr_active_pipeline.computation_results['maze'].

In [ ]:
from ansi2html import Ansi2HTMLConverter # used by DocumentationFilePrinter to build html document from ansi-color coded version
from pyphocorehelpers.print_helpers import DocumentationFilePrinter

doc_printer = DocumentationFilePrinter(doc_output_parent_folder=Path('C:/Users/pho/repos/PhoPy3DPositionAnalysis2021/EXTERNAL/DEVELOPER_NOTES/DataStructureDocumentation'), doc_name='PfND')
doc_printer.save_documentation('PfND', active_pf_1D, non_expanded_item_keys=['_reverse_cellID_index_map'])

In [29]:
import matplotlib.pyplot as plt
%matplotlib qt

In [ ]:
#
## Test that changing the position bins post-hoc is equivalent to initially computing with those position bins
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.analyses.placefields import PfND
from neuropy.utils.debug_helpers import debug_print_placefield, debug_print_subsession_neuron_differences
from neuropy.utils.debug_helpers import debug_print_ratemap, debug_print_spike_counts, debug_plot_2d_binning

spikes_df = curr_active_pipeline.sess.spikes_df
active_pos = curr_active_pipeline.sess.position

In [ ]:
curr_active_pipeline.active_sess_config

In [21]:
# all spikes:
active_epoch_placefields1Da = PfND(deepcopy(spikes_df), deepcopy(active_pos.linear_pos_obj), grid_bin=(1,1)) # 

In [ ]:
# pyramidal spikes only:
active_epoch_placefields1Db = PfND(deepcopy(spikes_df).spikes.sliced_by_neuron_type('pyramidal'), deepcopy(active_pos.linear_pos_obj), grid_bin=(1,1)) # grid_bin=, etc

In [ ]:
# PlacefieldComputationParameters
# Parameter sweeps:

# grid_bin = [(1.0, 1.0)]

# 10.0
cls.compute_position_grid_bin_size(sess.position.x, sess.position.y, num_bins=(64, 64)

In [37]:
smooth_options = [(None, None), (0.5, 0.5), (1.0, 1.0), (2.0, 2.0), (5.0, 5.0)]
output_pfs = {} # empty dict

for smooth in smooth_options:
    # pyramidal spikes only:
    output_pfs[smooth] = PfND(deepcopy(spikes_df).spikes.sliced_by_neuron_type('pyramidal'), deepcopy(active_pos.linear_pos_obj), grid_bin=(1,1), smooth=smooth, speed_thresh=10.0) # grid_bin=, etc

In [38]:
# Plot the differences:
fig, axs = plt.subplots(1,5);
plt.subplots_adjust(top=0.968,bottom=0.05,left=0.021,right=0.993,hspace=0.2,wspace=0.116)
for i, smooth in enumerate(smooth_options):
    output_pf = output_pfs[smooth]
    output_pf.plot_ratemaps_1D(ax=axs[i])
    axs[i].set_title('') # TODO: display the parameter value without losing the number of good cells for each.
    debug_print_placefield(output_pf)
    
    

In [126]:
import itertools
from typing import List, Tuple, NamedTuple

def parameter_sweeps(**kwargs):
    """ Returns every unique combination of the passed in parameters. Superior to cartesian_product as it preserves the labels (returning a flat list of dicts) and accepts more than 2 inputs.
    
    Usage:
        all_param_sweep_options, param_sweep_option_n_values = parameter_sweeps(smooth=[(None, None), (0.5, 0.5), (1.0, 1.0), (2.0, 2.0), (5.0, 5.0)], grid_bin=[(1,1),(5,5),(10,10)])
        >> all_param_sweep_options:  [{'smooth': (None, None), 'grid_bin': (1, 1)}, {'smooth': (None, None), 'grid_bin': (5, 5)}, {'smooth': (None, None), 'grid_bin': (10, 10)},
        {'smooth': (0.5, 0.5), 'grid_bin': (1, 1)},  {'smooth': (0.5, 0.5), 'grid_bin': (5, 5)},  {'smooth': (0.5, 0.5), 'grid_bin': (10, 10)},
        {'smooth': (1.0, 1.0), 'grid_bin': (1, 1)}, ...]
        >> param_sweep_option_n_values: {'smooth': 5, 'grid_bin': 3}
        
    NOTE:
        Replaces:
        all_param_sweep_options = cartesian_product(grid_bin_options, smooth_options)
        param_sweep_option_n_values = dict(grid_bin=len(grid_bin_options), smooth=len(smooth_options)) 

    """
    all_param_sweep_options = []
    param_sweep_option_n_values = {k:len(v) for k, v in kwargs.items()}
    for values in itertools.product(*kwargs.values()):
        all_param_sweep_options.append(dict(zip(kwargs.keys(), values))) # Output dictionary
        # param_sweep = ParamSweepNamedTuple(params=values)
        # all_param_sweep_options.append(param_sweep)
    return all_param_sweep_options, param_sweep_option_n_values
                      

def _compute_parameter_sweep(all_param_sweep_options):
    """ Computes the PfNDs for all the swept parameters (combinations of grid_bin, smooth, etc)
    
    Usage:
        smooth_options = [(None, None), (0.5, 0.5), (1.0, 1.0), (2.0, 2.0), (5.0, 5.0)]
        grid_bin_options = [(1,1),(5,5),(10,10)]
        all_param_sweep_options = cartesian_product(smooth_options, grid_bin_options)
        param_sweep_option_n_values = dict(smooth=len(smooth_options), grid_bin=len(grid_bin_options)) 
        output_pfs = _compute_parameter_sweep(all_param_sweep_options)

    """
    output_pfs = {} # empty dict

    for a_sweep_tuple in all_param_sweep_options:
        # pyramidal spikes only:
        if isinstance(a_sweep_tuple, dict):
            a_sweep_dict = a_sweep_tuple # already a dict
            a_sweep_tuple = frozenset(a_sweep_dict.items())
            
        else:
            # otherwise assume it's a named-tuple that can be converted to dict via a_sweep_tuple._asdict()
            a_sweep_dict = a_sweep_tuple._asdict()
            
        output_pfs[a_sweep_tuple] = PfND(deepcopy(spikes_df).spikes.sliced_by_neuron_type('pyramidal'), deepcopy(active_pos.linear_pos_obj), **a_sweep_dict) # grid_bin=, etc
        
    return output_pfs

def _plot_parameter_sweep(output_pfs, param_sweep_option_n_values):
    """ Sweeps a 1D parameter for the placefields and plots it
    
    Usage:
        fig, axs = _plot_parameter_sweep(output_pfs, param_sweep_option_n_values)
        
    """
    if len(output_pfs)>0:        
        if len(param_sweep_option_n_values) > 1:
            # more than one variable
            num_rows = list(param_sweep_option_n_values.values())[1] # get the first length
        else:
            # only one variable
            num_rows = 1
        
        num_columns = len(output_pfs) // num_rows
        
        # remove any singleton variables
        formatting_included_items_list = [k for k, v in param_sweep_option_n_values.items() if v>1]

        def _plot_title_formatter(x):
            # printable_dict = {k:v for k, v in x.items() if k in formatting_included_items_list} # if `x` is dict and has .items() method
            printable_dict = {k:v for k, v in x if k in formatting_included_items_list}
            return f"{printable_dict}"
        
        plot_title_formatter = _plot_title_formatter
        
        # plot_title_formatter = lambda x: f"{x}" # full NamedTuple item
        
        # # old namedtuple implementation:
        # if num_rows < 2:
        #     # suppress the constant variable in the title
        #     plot_title_formatter = lambda x: f"{x._fields[1]}={x[1]}" # only get the first item
        # else:
        #     plot_title_formatter = lambda x: f"{x}" # full NamedTuple item
        
    else:
        return # empty
    
    print(f'{num_rows = }, {num_columns = }')
    fig, axs = plt.subplots(num_rows, num_columns, sharex=True);
    plt.subplots_adjust(top=0.968,bottom=0.05,left=0.021,right=0.993,hspace=0.2,wspace=0.116)
    # Flatten the axs array
    axs = axs.ravel()
    for i, (param_sweep_tuple, output_pf) in enumerate(output_pfs.items()):
        output_pf.plot_ratemaps_1D(ax=axs[i])
        axs[i].set_title(plot_title_formatter(param_sweep_tuple)) # TODO: display the parameter value without losing the number of good cells for each.
        debug_print_placefield(output_pf)
    return fig, axs

In [104]:

# all_param_sweep_options = cartesian_product(grid_bin_options, smooth_options)
all_param_sweep_options, param_sweep_option_n_values = parameter_sweeps(smooth=[(None, None), (0.5, 0.5), (1.0, 1.0), (2.0, 2.0), (5.0, 5.0)], grid_bin=[(1,1),(5,5),(10,10)])
all_param_sweep_options

[{'smooth': (None, None), 'grid_bin': (1, 1)},
 {'smooth': (None, None), 'grid_bin': (5, 5)},
 {'smooth': (None, None), 'grid_bin': (10, 10)},
 {'smooth': (0.5, 0.5), 'grid_bin': (1, 1)},
 {'smooth': (0.5, 0.5), 'grid_bin': (5, 5)},
 {'smooth': (0.5, 0.5), 'grid_bin': (10, 10)},
 {'smooth': (1.0, 1.0), 'grid_bin': (1, 1)},
 {'smooth': (1.0, 1.0), 'grid_bin': (5, 5)},
 {'smooth': (1.0, 1.0), 'grid_bin': (10, 10)},
 {'smooth': (2.0, 2.0), 'grid_bin': (1, 1)},
 {'smooth': (2.0, 2.0), 'grid_bin': (5, 5)},
 {'smooth': (2.0, 2.0), 'grid_bin': (10, 10)},
 {'smooth': (5.0, 5.0), 'grid_bin': (1, 1)},
 {'smooth': (5.0, 5.0), 'grid_bin': (5, 5)},
 {'smooth': (5.0, 5.0), 'grid_bin': (10, 10)}]

In [107]:
# smooth_options = [(None, None), (0.5, 0.5), (1.0, 1.0), (2.0, 2.0), (5.0, 5.0)]
# grid_bin_options = [(1,1),(5,5),(10,10)]
# grid_bin_options = [(5,5)]
# param_sweep_option_n_values = dict(smooth=len(smooth_options), grid_bin=len(grid_bin_options)) 

smooth_options = [(None, None)]
grid_bin_options = [(0.5, 0.5), (1.0, 1.0), (2.0, 2.0), (5.0, 5.0)]
# all_param_sweep_options = cartesian_product(grid_bin_options, smooth_options)
# param_sweep_option_n_values = dict(grid_bin=len(grid_bin_options), smooth=len(smooth_options)) 
all_param_sweep_options, param_sweep_option_n_values = parameter_sweeps(smooth=smooth_options, grid_bin=grid_bin_options)
output_pfs = _compute_parameter_sweep(all_param_sweep_options)

In [128]:
speed_thresh_options = [25.0, 50.0, 100.0, 200.0]
# all_param_sweep_options = cartesian_product(speed_thresh_options, smooth_options)
# param_sweep_option_n_values = dict(speed_thresh=len(speed_thresh_options), smooth=len(smooth_options)) 
all_param_sweep_options, param_sweep_option_n_values = parameter_sweeps(speed_thresh=speed_thresh_options)
output_pfs = _compute_parameter_sweep(all_param_sweep_options)

In [129]:
fig, axs = _plot_parameter_sweep(output_pfs, param_sweep_option_n_values)

num_rows = 1, num_columns = 4
good_placefield_neuronIDs: (56 good)
num_spikes: (24761 total spikes)
good_placefield_neuronIDs: (44 good)
num_spikes: (11724 total spikes)


ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
active_epoch_placefields1Da.plot_ratemaps_1D()

In [ ]:
active_epoch_placefields1Db.cell_ids

In [ ]:
active_config.computation_config = PlacefieldComputationParameters(speed_thresh=0.0, grid_bin=(5, 3), smooth=(0.0, 0.0), frate_thresh=0.1) # TODO: FIXME: BUG: when frate_thresh=0.0, there are 0 good placefield_neuronIDs for all computations!

In [ ]:
## All Spikes:
active_epoch_session = sess.filtered_by_neuron_type('pyramidal').filtered_by_epoch(active_epoch)
print_subsession_neuron_differences(sess.neurons, active_epoch_session.neurons)
# print(sess.neurons.n_spikes)

# # ## Lap_specific Spikes Only:
# active_lap_specific_epoch_session = lap_specific_session.filtered_by_neuron_type('pyramidal').filtered_by_epoch(active_epoch)
# print_subsession_neuron_differences(lap_specific_session.neurons, active_lap_specific_epoch_session.neurons)
# # print(active_lap_specific_epoch_session.neurons.n_spikes)

## Configure Placefield Calc:
should_display_2D_plots = False

# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=2, smooth=0.5, frate_thresh=2.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=10, smooth=0.5, frate_thresh=2.0) # works well
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=2.5, smooth=1.5, frate_thresh=2.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=(10, 3), smooth=(0.5, 0.5), frate_thresh=0.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=0.0, grid_bin=(3, 4), smooth=(2, 1), frate_thresh=2.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=(10, 10), smooth=(0.5, 0.5), frate_thresh=2.0) ## Works well for 2D Placemaps
# height: 20.0
# width: 250.0
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=0, grid_bin=(2.0, 0.2), smooth=(0.5, 0.5), frate_thresh=2.0) ## Extremely Slow
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=0, grid_bin=(2.0, 1.0), smooth=(0.5, 0.5), frate_thresh=2.0) ## Very slow, doesn't work

# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=(10, 3), smooth=(0.0, 0.0), frate_thresh=2.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=(10, 3), smooth=(0.1, 0.1), frate_thresh=2.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=(10, 3), smooth=(1.0, 10.0), frate_thresh=2.0)

# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=0.0, grid_bin=(25, 9), smooth=(0.0, 0.0), frate_thresh=2.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=0.0, grid_bin=(5, 3), smooth=(0.0, 0.0), frate_thresh=2.0)
active_config.computation_config = PlacefieldComputationParameters(speed_thresh=0.0, grid_bin=(5, 3), smooth=(0.0, 0.0), frate_thresh=0.1) # TODO: FIXME: BUG: when frate_thresh=0.0, there are 0 good placefield_neuronIDs for all computations!


active_epoch_placefields1D, active_epoch_placefields2D = compute_placefields_masked_by_epochs(active_epoch_session, active_config, included_epochs=None, should_display_2D_plots=should_display_2D_plots)
even_lap_specific_placefields1D, even_lap_specific_placefields2D = compute_placefields_masked_by_epochs(active_epoch_session, active_config, included_epochs=even_lap_specific_epochs, should_display_2D_plots=should_display_2D_plots)
odd_lap_specific_placefields1D, odd_lap_specific_placefields2D = compute_placefields_masked_by_epochs(active_epoch_session, active_config, included_epochs=odd_lap_specific_epochs, should_display_2D_plots=should_display_2D_plots)
any_lap_specific_placefields1D, any_lap_specific_placefields2D = compute_placefields_masked_by_epochs(active_epoch_session, active_config, included_epochs=any_lap_specific_epochs, should_display_2D_plots=should_display_2D_plots)
# Compare the results
# debug_print_ratemap(active_epoch_placefields1D.ratemap)
# num_spikes_per_spiketrain = np.array([np.shape(a_spk_train)[0] for a_spk_train in active_epoch_placefields1D.spk_t])
# num_spikes_per_spiketrain
# print('placefield_neuronID_spikes: {}; ({} total spikes)'.format(num_spikes_per_spiketrain, np.sum(num_spikes_per_spiketrain)))

debug_print_placefield(active_epoch_placefields1D)

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BayesianPlacemapPositionDecoder

## Build the new decoder with custom params:
active_computation_config = curr_active_pipeline.computation_results[active_config_name].computation_config
new_decoder_pf_params = deepcopy(active_computation_config.pf_params) # should be a PlacefieldComputationPcurr_active_pipeliners
# override some settings before computation:
# new_decoder_pf_params.time_bin_size = time_bin_size

## 1D Decoder
new_decoder_pf1D = active_pf_1D
new_1D_decoder_spikes_df = new_decoder_pf1D.filtered_spikes_df.copy()

# Why would it need both the pf1D and the spikes? Doesn't the pf1D include the spikes (and determine the placefields, which are all that are used)???
new_1D_decoder = BayesianPlacemapPositionDecoder(new_decoder_pf_params.time_bin_size, new_decoder_pf1D, new_1D_decoder_spikes_df, debug_print=False)

new_1D_decoder.compute_all() #  --> n = self.


print(f'done!')

In [ ]:
%pdb off

In [ ]:
# _perform_specific_epochs_decoding

In [ ]:
curr_active_pipeline.perform_specific_computation(

# 2023-02-27 - Test whether conform to active position works

In [ ]:

from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BayesianPlacemapPositionDecoder, Zhang_Two_Step

prev_output_result.computed_data['pf1D_Decoder'] = BayesianPlacemapPositionDecoder(pf_computation_config.time_bin_size, prev_output_result.computed_data['pf1D'], prev_output_result.computed_data['pf1D'].filtered_spikes_df.copy(), debug_print=False)
prev_output_result.computed_data['pf1D_Decoder'].compute_all() #

prev_output_result.computed_data['pf2D_Decoder'] = BayesianPlacemapPositionDecoder(pf_computation_config.time_bin_size, prev_output_result.computed_data['pf2D'], prev_output_result.computed_data['pf2D'].filtered_spikes_df.copy(), debug_print=False)
prev_output_result.computed_data['pf2D_Decoder'].compute_all() #